In [2]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.functions import KernelArguments

from dotenv import load_dotenv
import os


In [3]:
kernel = Kernel()

load_dotenv()
kernel.add_service(
    AzureChatCompletion(service_id="groundness", 
                        deployment_name=os.getenv("DEPLOYMENT_NAME"), 
                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
                        api_key=os.getenv("AZURE_OPENAI_API_KEY"))
)

In [4]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant's house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort's coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife."""


In [7]:
groundingSemanticFunctions = kernel.add_plugin(parent_directory="C:\\Users\\CheekireddyDhaminiMA\\.vscode\\semantic-kernel\\prompt_template_samples",
plugin_name="GroundingPlugin")

In [8]:
entity_extraction = groundingSemanticFunctions["ExtractEntities"]
reference_check = groundingSemanticFunctions["ReferenceCheckEntities"]
entity_excision = groundingSemanticFunctions["ExciseEntities"]

In [9]:
summary_text = """
My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of
misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out,
finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and
his daughter, Mary. Mary procured work to eke out a living, but after ten months her father died, leaving
her a beggar. My father came to her aid and two years later they married when they visited Rome.
"""

summary_text = summary_text.replace("\n", " ").replace("  ", " ")
print(summary_text)

 My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out, finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and his daughter, Mary. Mary procured work to eke out a living, but after ten months her father died, leaving her a beggar. My father came to her aid and two years later they married when they visited Rome. 


In [10]:
extraction_result = await kernel.invoke(
    entity_extraction,
    input=summary_text,
    topic="people and places",
    example_entities="John, Jane, mother, brother, Paris, Rome",
)

print(extraction_result)

<entities>
- father (the narrator's father; a person)
- Milan (a city in Italy; a place)
- Beaufort (a merchant and friend of the narrator's father; a person)
- Zurich (a city in Switzerland; a place)
- Mary (Beaufort's daughter; a person)
- Rome (a city in Italy; a place)
</entities>


In [11]:
grounding_result = await kernel.invoke(
    reference_check, input=str(extraction_result.value), reference_context=grounding_text
)

print(grounding_result)

<ungrounded_entities>
- Milan (a city in Italy; a place)
- Zurich (a city in Switzerland; a place)
- Mary (Beaufort's daughter; a person)
- Rome (a city in Italy; a place)
</ungrounded_entities>


In [12]:
excision_result = await kernel.invoke(
    entity_excision, input=summary_text, ungrounded_entities=str(grounding_result.value)
)

print(excision_result)

My father, a respected resident, was a close friend of a merchant named Beaufort who, after a series of misfortunes, moved away in poverty. My father was upset by his friend's troubles and sought him out, finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and his daughter. She procured work to eke out a living, but after ten months her father died, leaving her a beggar. My father came to her aid and two years later they married when they visited another city.


##### Streaming results - This is used to delivery the response token-by-toke, rather than waiting for the entire block of the para to generate. 

In [13]:
oai_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="oai_text",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

In [14]:
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

aoai_chat_service  = AzureChatCompletion(service_id="aoai_chat", 
                                         deployment_name=os.getenv("DEPLOYMENT_NAME"),
                                         endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                         api_key=os.getenv("AZURE_OPENAI_API_KEY"))

aoai_text_service = AzureTextCompletion(service_id="aoai_text", 
                                        deployment_name=os.getenv("DEPLOYMENT_NAME"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"))

prompt = "provide me a list of possible meaning for the acroym ORLD."
stream = aoai_text_service.get_streaming_text_contents(prompt=prompt, settings=oai_prompt_execution_settings)

C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_31964\911685289.py:8: DeprecationWarning: The AzureTextCompletion class is deprecated and will be removed after 01/01/2026. There won't be a replacement because all text completion models on Azure OpenAI have retired. Please migrate to chat completion models before the deprecation date.
  aoai_text_service = AzureTextCompletion(service_id="aoai_text",


In [17]:
from semantic_kernel.contents import ChatHistory

content = "You are an AI assistant that helps people find information."
chat = ChatHistory()
chat.add_system_message(content)
chat.add_user_message('Why is the existance of species is important?')
stream = aoai_chat_service.get_streaming_chat_message_contents(
    chat_history=chat, settings=oai_prompt_execution_settings
)

async for text in stream:
    print(str(text[0]), end="")

The existence of species is important for several reasons:

1. **Biodiversity and Ecosystem Health:** Each species plays a unique role in its ecosystem, contributing to processes like pollination, nutrient cycling, pest control, and decomposition. Healthy ecosystems with diverse species are more resilient to changes and disturbances.

2. **Balance in Food Webs:** Species interact through food webs, maintaining balance by controlling populations of other organisms. Removing one species can lead to imbalances that affect many others.

3. **Human Benefits:** Many species provide direct benefits to humans, such as food, medicine, clothing, and building materials. For example, plants used for medicines or crops for food are essential for human survival.

4. **Scientific Knowledge:** Studying different species

###### Agent 

In [ ]:
from azure.identity.aio import AzuureCliCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

USER_INPUTS = [
    "How can you help me?"
]
